# Camada Delivery
Adaptação dos dados para consumo

## dlzd_olist_order_reviews
Transformações baseadas na camada Trusted para:
* Obter tokens a partir dos textos de review_comment_title e review_comment_message
* Gerar nova coluna review_comment_title_and_message com os tokens do título e comentário de uma mesma avaliação
* Padronizar tokens:
  * Lower case
  * Remover palavras comuns e conectores
  * Desconsiderar de palavras com 1 caractere e mais de 15 caracteres
  * Desconsiderar de espaços em branco, números e caracteres especiais


Importando bibliotecas necessárias

In [1]:
from gensim.utils import simple_preprocess
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada trusted

In [3]:
dlzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Utilizando o pacote *simple_preprocess*, responsável por gerar uma lista de tokens

Tranformações aplicadas:
*   Padronização em lower case
*   Desconsideração de palavras com 1 caracter e mais de 15 caracteres
*   Desconsideração de espaços em branco, números e caracteres especiais


Doc: https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [5]:
dlzd_olist_order_reviews['review_comment_title'] = dlzd_olist_order_reviews['review_comment_title'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

dlzd_olist_order_reviews['review_comment_message'] = dlzd_olist_order_reviews['review_comment_message'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

Unindo título com comentários

In [6]:
titles = dlzd_olist_order_reviews['review_comment_title']
messages = dlzd_olist_order_reviews['review_comment_message']

dlzd_olist_order_reviews['review_comment_title_and_message'] = titles + messages

Reorganizando ordem das colunas

In [7]:
dlzd_olist_order_reviews = dlzd_olist_order_reviews.loc[:,
    [
        'review_id',
        'order_id',
        'review_score',
        'review_comment_title',
        'review_comment_message',
        'review_comment_title_and_message',
        'review_creation_date',
        'review_answer_timestamp'
    ]
]

Visualizando dataset transformado

In [9]:
dlzd_olist_order_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
23397,0e7bc73fde6782891898ea71443f9904,bd78f91afbb1ecbc6124974c5e813043,4,[],[aprovado],[aprovado],2018-07-04,2018-07-05 00:25:13
23398,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,"[muito, bom, produto]","[ficamos, muito, satisfeitos, com, produto, atende, as, nossas, necessidades, com, muita, facilidade, recomendo]","[muito, bom, produto, ficamos, muito, satisfeitos, com, produto, atende, as, nossas, necessidades, com, muita, facilidade, recomendo]",2018-06-30,2018-07-02 23:09:35
23399,40743b46a0ee86375cedb95e82b78d75,3e93213bb8fdda91186b4018b2fe0030,5,"[otima, embalagem]",[],"[otima, embalagem]",2018-08-08,2018-08-08 16:56:16
23400,2ee221b28e5b6fceffac59487ed39348,f2d12dd37eaef72ed7b1186b2edefbcd,2,"[foto, enganosa]","[foto, muito, diferente, principalmente, graninha, sintetica, muito, rala, diferente, da, foto]","[foto, enganosa, foto, muito, diferente, principalmente, graninha, sintetica, muito, rala, diferente, da, foto]",2018-03-28,2018-05-25 01:23:26
23401,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,[],"[produto, nao, foi, enviado, com, nf, nao, existe, venda, sem, nf, com, certeza, fico, no, aguardo, do, envio, da, nf, podendo, ser, por, mail]","[produto, nao, foi, enviado, com, nf, nao, existe, venda, sem, nf, com, certeza, fico, no, aguardo, do, envio, da, nf, podendo, ser, por, mail]",2018-05-19,2018-05-20 21:51:06


Salvando dados na camada delivery em formato parquet

In [10]:
dlzd_olist_order_reviews.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

## dlzd_olist_order_reviews_clean
Transformações baseadas na tabela dlzd_olist_order_reviews para remoção de stop words presentes nos tokens das colunas:
* review_comment_title
* review_comment_message
* review_comment_title_and_message


Importação das bibliotecas

In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_sm

In [17]:
from gensim.utils import simple_preprocess
import pandas as pd
import spacy

Leitura da tabela dlzd_olist_order_reviews

In [18]:
dlzd_olist_order_reviews_clean = pd.read_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Remoção de stop words

In [20]:
# conjunto de stop words a partir do modelo SpaCy
nlp = spacy.load('pt_core_news_sm')
stop_words = set(nlp.Defaults.stop_words)

In [19]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

In [21]:
dlzd_olist_order_reviews_clean['review_comment_title_and_message'] = dlzd_olist_order_reviews_clean['review_comment_title_and_message'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean['review_comment_message'] = dlzd_olist_order_reviews_clean['review_comment_message'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean['review_comment_title'] = dlzd_olist_order_reviews_clean['review_comment_title'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,[],"[recebi, prazo, estipulado]","[recebi, prazo, estipulado]",2017-04-21,2017-04-21 22:02:06
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,[],"[parabens, lojas, lannister, adorei, comprar, internet, seguro, pratico, parabens, feliz, pascoa]","[parabens, lojas, lannister, adorei, comprar, internet, seguro, pratico, parabens, feliz, pascoa]",2018-03-01,2018-03-02 10:26:53
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,[recomendo],"[aparelho, eficiente, site, marca, aparelho, impresso, desinfector, chegar, outro, nome, atualizar, marca, correta, aparelho]","[recomendo, aparelho, eficiente, site, marca, aparelho, impresso, desinfector, chegar, outro, nome, atualizar, marca, correta, aparelho]",2018-05-22,2018-05-23 16:45:47
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,[],"[travando, ta]","[travando, ta]",2018-02-16,2018-02-20 10:52:22
4,9314d6f9799f5bfba510cc7bcd468c01,0dacf04c5ad59fd5a0cc1faa07c34e39,2,[],"[gostaria, houve, recebi, compra, decpcionou]","[gostaria, houve, recebi, compra, decpcionou]",2018-01-18,2018-01-20 21:25:45
...,...,...,...,...,...,...,...,...
35754,98fffa80dc9acbde7388bef1600f3b15,d398e9c82363c12527f71801bf0e6100,4,[],"[produto, recebi, acordo, compra, realizada]","[produto, recebi, acordo, compra, realizada]",2017-11-29,2017-11-30 15:52:51
35755,df5fae90e85354241d5d64a8955b2b09,509b86c65fe4e2ad5b96408cfef9755e,5,[],"[entregou, prazo, produto, chegou, condicoes, perfeitas, satisfeita]","[entregou, prazo, produto, chegou, condicoes, perfeitas, satisfeita]",2018-02-07,2018-02-19 19:47:23
35756,a709d176f59bc3af77f4149c96bae357,d5cb12269711bd1eaf7eed8fd32a7c95,3,[],"[produto, nao, enviado, nf, nao, existe, venda, nf, fico, aguardo, envio, nf, podendo, mail]","[produto, nao, enviado, nf, nao, existe, venda, nf, fico, aguardo, envio, nf, podendo, mail]",2018-05-19,2018-05-20 21:51:06
35757,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,[],"[solicitei, compra, capa, retrovisor, celta, prisma, meriva, preta, esquerdo, chegou, mim, capa, direito, outro, modelo, carro, decepcionado, fico, aguardo]","[solicitei, compra, capa, retrovisor, celta, prisma, meriva, preta, esquerdo, chegou, mim, capa, direito, outro, modelo, carro, decepcionado, fico, aguardo]",2017-12-15,2017-12-16 01:29:43


In [ ]:
dlzd_olist_order_reviews_clean.count()

,0
review_id,35759
order_id,35759
review_score,35759
review_comment_title,35759
review_comment_message,35759
review_comment_title_and_message,35759
review_creation_date,35759
review_answer_timestamp,35759


Salvamento de arquivo parquet na camada delivery

In [22]:
dlzd_olist_order_reviews_clean.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews_clean.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)